Тег - короткое слово, описывающее одну из характеристик товара. У товара может быть несколько тегов, один тег может быть у нескольких товаров. Мы задаем множество тегов, стараясь включить теги для описания любых товаров на площадке. Естественно, у нас не хватает знания во всех предметных областях, поэтому сразу наших тегов не будет достаточно. Их можно будет расширять по запросу продавцов при одобрении модератора.

Категория - множество тегов, один из которых описывает товар. Например, существует корневая категория, включающая теги "авто", "одежда", "продукты"(для примера 3 тега, в реальности их больше). У каждого тега могут быть дочерние категории. Это значит, что если товар имеет этот тег, он может иметь теги из дочерних категорий. Например, товар, имеющий тег "одежда" имеет дочернюю категорию "ткань"
товар с тегом "авто" имеет дочернюю категорию "марка авто". Сложно вручную за короткое время выявить все возможные отношения тегов, но выявление этих отношений должно облегчить поиск товаров и получение тегов из запроса, а в перспективе можно улучшить существующие отношения.

"но это же просто категории!" - такие теги более гибкие чем простые категории, они облегчают поиск и позволяют задавать теги из одной категории совершенно разным товарам, упрощая реализацию.

### udp 0.1
- товар может иметь несколько тегов из категории (например набор из товаров разного цвета)
- возможность рекурсивно задавать категории тегов не ограничивается (иначе это была бы строгая иерархия). Адекватность задания категорий - ответственность на продавца.
- тег может относиться к нескольким категориям: один бренд может иметь товары в разных сферах. Так, многие производители электроники производят продукцию разных секторов. Иначе пришлось бы делать единую категорию "бренд" со всеми производителями всех категорий. *может можно придумать что-то лучше?*. Тег "кисти" есть в инструментах и в канцелярии, поиск должен работать по всем тегам.
- naming style. Тег предпочтительно должен описывать множество: это должно быть слово во множественном числе или слово, подразумевающее множество вещей.

In [22]:
from dataclasses import dataclass,field

@dataclass
class tag:
    id: int
    name: str
    subsidiary_categories: list['category'] = field(default_factory=list)

@dataclass
class category:
    id: int
    name: str
    tags: dict[str,tag] =field(default_factory=dict)
    
    def add_tags(self,tag_names:list[str]):
        for tag_name in tag_names:
            self.tags[tag_name]=tag(tag_name)
    def add_subsidiary_to_tags(self,subsidiary: 'category',tag_names:list[str]):
        for tag_name in tag_names:
            cats = self.tags[tag_name].subsidiary_categories
            if subsidiary not in cats:
                cats.append(subsidiary)
                


авто одежда обувь хобби и отдых потребительская электроника животные фармацевтика красота кулинария аксессуары дом инструменты канцелярия 

In [56]:
# tags demo
root = category("корневая категория")

root.add_tags(["одежда","обувь","хобби и отдых","потребительская электроника","животные","аптека","красота","кулинария","аксессуары","дом","инструменты","канцелярия"])

sex = category("пол")
sex.add_tags(["мужчины","женщины","унисекс"])
root.add_subsidiary_to_tags(sex,["одежда","обувь","аксессуары"])

color = category("цвет")
color.add_tags(["бежевый","белый","голубой","желтый","зеленый","коричневый","красный","оранжевый","розовый","серый","синий","розовый","черный"])
root.add_subsidiary_to_tags(color,["одежда","обувь"])


age = category("возрастная категория")
age.add_tags(["младенцы","дети","подростки","взрослые"])
root.add_subsidiary_to_tags(age,["одежда","обувь","хобби и отдых"])

writing = category("пишущие принадлежности")
writing.add_tags(["ручки","карандаши","маркеры","фломастеры","кисти","чернила","мел","мелки"])
root.add_subsidiary_to_tags(writing,["канцелярия"])
root.add_subsidiary_to_tags(color, ["канцелярия"])

pencil_stiffness = category("твердость карандаша")
pencil_stiffness.add_tags(["9H","8H","7H","6H","5H","4H","3H","2H","H","F","HB","B","2B","3B","4B","5B","6B","7B","8B","9B"])
writing.add_subsidiary_to_tags(pencil_stiffness,["карандаши"])

for t in root.tags:
    print(t,end=' ')

одежда обувь хобби и отдых потребительская электроника животные фармацевтика красота кулинария аксессуары дом инструменты канцелярия 

In [87]:
# visualization

from pyvis.network import Network
import networkx as nx

g = Network(notebook=False,directed=True,height='750px', width='100%',
                  bgcolor='#222222',font_color='white')


def draw_graph_dfs(g: Network,root:category,ids:dict[str,int]={},counter:list[int]=[0]):
    
    def get_id(name:str)->int:
        id = ids.get(name)
        if id is None:
            id = counter[0]
            counter[0]+=1
            ids[name]=id
        return id
    
    if len(ids)==0:
        id = counter[0]
        g.add_node(id,root.name,color="gold",size=50)
        ids[root.name]=id
        counter[0]+=1

    for t in root.tags.values():
        id = get_id(t.name)
        g.add_node(id,t.name,size=10,color="#A2DA5A")
        g.add_edge(ids[root.name],id,length=50)
        
        for cat in t.subsidiary_categories:
            id = get_id(cat.name)
            g.add_node(id,cat.name,size=20,color="#A877C8",arrowStrikethrough=True)
            g.add_edge(ids[t.name],id)
            draw_graph_dfs(g,cat,ids,counter)




In [88]:
# show demo 

draw_graph_dfs(g,root)
g.show("tag_graph.html")

In [ ]:
# full tag list

root.add_tags(["авто"])
root.add_subsidiary_to_tags(color,["авто"])

auto_brand = category("автомобильная марка")
auto_brand.add_tags(["Abadal","Abarth","Abbott-Detroit","ABT","AC Cars","Acura","Agrale","Aiways","Aixam","Alfa Romeo","Alpina","Alpine","Alta","Alvis","AMC","AMG","Apollo","Arash","Ariel","ARO","Arrinera","Arrival","Artega","Ascari","Askam","Aston Martin","Atalanta Motors","Auburn","Audi","Audi Sport","Aurus","Austin","Autobacs","Autobianchi","Axon","BAC","BAIC Motor","Baltijas Džips","Baojun","Bentley","Berkeley","Berliet","Bertone","BharatBenz","Bitter","Bizzarrini","BMW","Borgward","Bowler","Brabus","Brammo","Brilliance","Bristol","Bronto","Brooke","Bufori","Bugatti","Buick","BYD","Cadillac","Caparo","Carlsson","Caterham","Changan","Changfeng","Chery","Chevrolet","Chrysler","Cisitalia","Citroën","Cizeta","Cole","Corre La Licorne","Corvette","Dacia","Daewoo","DAF","Daihatsu","Daimler","Dartz","Datsun","David Brown","De Tomaso","Delage","DeLorean (DMC)","Detroit Electric","Devel Motors","Diatto","DINA","DKW","Dodge","Dongfeng","Donkervoort","Drive eO","DS","Eagle","EDAG","Edsel","Eicher","Elemental","Elfin","Elva","Englon","ERF","Eterniti","Ё-Auto","Facel Vega","Faraday Future","FAW","Ferrari","Fiat","Fioravanti","Fisker","Foden Trucks","Force Motors","Ford","Foton","FPV","Franklin","Freightliner","GAC Group","Gardner Douglas","GAZ","Geely","General Motors","Genesis","Geo","Gilbern","Gillet","Ginetta","GMC","Gonow","Great Wall","Grinnall","GT-R","GTA Motor (Spania GTA)","Gumpert","Hafei","Haima","Haval","Hawtai","Hennessey","Hillman","Hindustan Motors","Hino","Hispano-Suiza","Holden","Hommell","Honda","Horch","HSV","Hudson","Hummer","Hupmobile","Hyundai","IC Bus","Infiniti","Innocenti","Intermeccanica","International Harvester","International Trucks","Iran Khodro","Irizar","Isdera","Iso Rivolta","Isuzu","Iveco","JAC","Jaguar","Jawa","JBA","Jeep","Jensen","JMC","Kaiser","Karma","Keating","Kenworth","Kia","Koenigsegg","KTM","Lada","Lagonda","Lamborghini","Lancia","Land Rover","Landwind","Laraki","LEVC","Lexus","Leyland","Lifan","Ligier","Lincoln","Lister","Lloyd","Lobini","Lotus","Lucid","Luxgen","M BMW","Mack","Mahindra","MAN","Mansory","Marcos","Marlin","Maserati","Mastretta","Maxus","Maybach","MAZ","Mazda","Mazzanti","McLaren","Melkus","Mercedes-Benz","Mercury","Merkur","MG","Microcar","Mills Extreme Vehicles","Mini","Mitsubishi","Mitsuoka","MK Sportscars","Monteverdi","Morgan","Morris","Mosler","Mustang","Navistar","Nismo","Nissan","Noble","Oldsmobile","Oltcit","Opel","OSCA","Paccar","Pagani","Panhard","Panoz","Pegaso","Perodua","Peterbilt","Peugeot","PGO","Pierce-Arrow","Pininfarina","Plymouth","Polestar","Pontiac","Porsche","Praga","Premier","Prodrive","Proton","Qoros","Radical Sportscars","RAM","Rambler","Ranz","Renault","Renault Samsung","Rezvani","Riley","Rimac","Rinspeed","Roewe","Rolls-Royce","Ronart Cars","Rossion","Rover","Ruf","Saab","SAIC Motor","SAIPA","Saleen","Saturn","Scania","Scion","SEAT","Setra","Shelby","Simca","Singer","Sisu Auto","Škoda","Smart","Soueast","Spirra","Spyker","SRT","SsangYong","SSC","Sterling","Studebaker","Subaru","Suffolk","Suzuki","Talbot","Tata","Tatra","Tauro Sport Auto","TechArt","Tesla","Thai Rung","Toroidion","Toyota","Toyota Crown","Tramontana","Trion","Triumph","Troller","TVR","UAZ","UD Trucks","Ultima Sports","Vandenbrink","Vauxhall","Vector Motors","Vencer","Venturi","Venucia","Viper","Volkswagen","Volvo","W Motors","Wanderer","Wartburg","Western Star","Westfield","Wiesmann","Willys-Overland","Wuling","XPeng","Yulon"])
root.add_subsidiary_to_tags(auto_brand,["авто"])